In [15]:
open_test.transcript[open_test.transcript.apply(lambda x:len(x.split()))==17]#.describe()

1320    THOSE LOVE PANGS RIVALS THERE'S A FEMALE HIDIN...
Name: transcript, dtype: object

In [5]:
from datasets import load_dataset
ted=load_dataset('csv',data_files={#'train':'/home/nxingyu/project/data/ted_talks_processed.train.csv',
                                    #     'dev':'/home/nxingyu/project/data/ted_talks_processed.dev.csv',
                                         'test':'/home/nxingyu/project/data/ted_talks_processed.test.csv'})
# subtitles=load_dataset('csv',data_files={'train':'/home/nxingyu/project/data/open_subtitles_processed.train.csv',
#                                          'dev':'/home/nxingyu/project/data/open_subtitles_processed.dev.csv',
#                                          'test':'/home/nxingyu/project/data/open_subtitles_processed.test.csv'})

Using custom data configuration default


Dataset csv downloaded and prepared to /home/nxingyu/.cache/huggingface/datasets/csv/default-a2ce7fdae4f5a4d8/0.0.0/2960f95a26e85d40ca41a230ac88787f715ee3003edaacb8b1f0891e9f04dda2. Subsequent calls will reuse this data.


In [6]:
import pandas as pd
def punct_proportion(df):
    l=[]
    st=pd.Series(df['transcript']).str
    for c in ".?!,;:-—…":
        l.append(sum(st.count("\\"+c)))
    [print(i[0],i[1],i[1]/sum(l)) for i in zip(list(".?!,;:-—…"),l)]
    print(st.count('\w+').sum())
for split in ted:
    punct_proportion(ted[split])
    print()

. 40835 0.3979476484690198
? 3679 0.035852807609098175
! 291 0.002835870349075175
, 48727 0.4748572319566531
; 586 0.005710721733876469
: 1243 0.012113356851891554
- 3874 0.03775313310074649
— 3168 0.03087298029508644
… 211 0.00205624963455279
734841



In [5]:
for split in subtitles:
    punct_proportion(subtitles[split])
    print()

. 47443035 0.44392753421469083
? 13250829 0.12398886041482206
! 6519047 0.0609991426589736
, 24551760 0.22973239965425646
; 57285 0.0005360194346227758
: 374276 0.003502124638437183
- 10479724 0.09805945244798349
— 9564 8.949096399986432e-05
… 4185605 0.03916497557221373
419201886

. 5921757 0.4430048481920479
? 1660096 0.12419127911939411
! 813038 0.06082312660995144
, 3062385 0.22909609462708524
; 8073 0.0006039386856729181
: 57019 0.004265574125899185
- 1321067 0.09882862227992877
— 591 4.421253105818092e-05
… 523225 0.03914230382896229
52422785

. 5921304 0.4422626972770535
? 1670946 0.12480309826421737
! 834767 0.06234881793231256
, 3063118 0.2287845428570959
; 7344 0.0005485239820152251
: 44933 0.0033560495756930976
- 1324557 0.09893127451608667
— 1210 9.037500248344532e-05
… 520479 0.03887462059304226
52341671



In [3]:
# ted_train.sort_values('talk_id').sample(frac=1,random_state=42).reset_index(drop=True)
ted_train=ted['train']
ted_dev=ted['dev']
ted_test=ted['test']
ted_train,ted_dev,ted_test

(Dataset({
     features: ['talk_id', 'transcript'],
     num_rows: 3195
 }),
 Dataset({
     features: ['talk_id', 'transcript'],
     num_rows: 399
 }),
 Dataset({
     features: ['talk_id', 'transcript'],
     num_rows: 400
 }))

In [7]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')
tags=sorted(list('.?!,;:-—…'))
tag2id = {tag: id+1 for id, tag in enumerate(tags)}
id2tag = {id: tag for tag, id in tag2id.items()}
tag2id,id2tag

({'!': 1, ',': 2, '-': 3, '.': 4, ':': 5, ';': 6, '?': 7, '—': 8, '…': 9},
 {1: '!', 2: ',', 3: '-', 4: '.', 5: ':', 6: ';', 7: '?', 8: '—', 9: '…'})

In [8]:
import torch
import regex as re
import numpy as np

tags=sorted(list('.?!,;:-—…'))
tag2id = {tag: id+1 for id, tag in enumerate(tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

def text2masks(n):
    def text2masks(text):
        '''Converts single paragraph of text into a list of words and corresponding punctuation based on the degree requested.'''
        if n==0: 
            refilter="(?<=[.?!,;:\-—… ])(?=[^.?!,;:\-—… ])|$" 
        else:
            refilter="[.?!,;:\-—…]{1,%d}(?= *[^.?!,;:\-—…]+|$)|(?<=[^.?!,;:\-—…]) +(?=[^.?!,;:\-—…])"%(n)
        word=re.split(refilter,text, flags=re.V1)
        punct=re.findall(refilter,text, flags=re.V1)
        wordlist,punctlist=([] for _ in range(2))
        for i in zip(word,punct+['']):
            w,p=i[0].strip(),i[1].strip()
            if w!='':
                wordlist.append(re.sub(r'[.?!,;:\-—… ]','',w))
                punctlist.append(0 if not w[-1] in '.?!,;:-—…' else tag2id[w[-1]])
            if p!='':
                wordlist.append(p)
                punctlist.append(0)
        return(wordlist,punctlist)
    return text2masks
assert(text2masks(0)('"Hello!!')==(['"Hello'], [1]))
assert(text2masks(1)('"Hello!!')==(['"Hello', '!'], [1, 0]))
assert(text2masks(0)('"Hello!!, I am human.')==(['"Hello','I','am','human'], [2,0,0,4]))
assert(text2masks(2)('"Hello!!, I am human.')==(['"Hello', '!,','I','am','human','.'], [1,0,0,0,0,0]))
def chunk_examples_with_degree(n):
    '''Ensure batched=True if using dataset.map or ensure the examples are wrapped in lists.'''
    def chunk_examples(examples):
        output={}
        output['texts']=[]
        output['tags']=[]
        for sentence in examples['transcript']:
            text,tag=text2masks(n)(sentence)
            output['texts'].append(text)
            output['tags'].append([0]+tag if text[0]!='' else tag) # [0]+tag so that in all case, the first tag refers to [CLS]
        return output
    return chunk_examples
assert(chunk_examples_with_degree(0)({'transcript':['Hello!Bye…']})=={'texts': [['Hello', 'Bye']], 'tags': [[0, 1, 9]]})

def encode_tags(encodings, docs, max_length, overlap):
    encoded_labels = []
    doc_id=0
    label_offset=1
#     print(encodings.keys())
    for doc_offset,current_doc_id in zip(encodings.offset_mapping,encodings['overflow_to_sample_mapping']):
#         print(doc_id, end=' ')
        if current_doc_id>doc_id:
            doc_id+=1
            label_offset=0
            print('.', end='')
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        doc_enc_labels[0]=docs[doc_id][label_offset-1] # Set leading punctuation class
#         print([id2tag[t] if t>0 else '' for t in docs[doc_id][label_offset:label_offset+len(doc_offset)]])
        arr_offset = np.array(doc_offset)
        arr_mask = (arr_offset[:,0] == 0) & (arr_offset[:,1] != 0) # Gives the labels that should be assigned punctuation
        doc_enc_labels[arr_mask] = docs[doc_id][label_offset:label_offset+sum(arr_mask)]
        encoded_labels.append(doc_enc_labels)            
        label_offset+=sum(arr_mask[:max_length-overlap-1])
    return encoded_labels

def process_dataset(dataset, split, max_length=128, overlap=63):
    data=dataset[split].map(chunk_examples_with_degree(0), batched=True, batch_size=max_length,remove_columns=dataset[split].column_names)
    encodings=tokenizer(data['texts'], is_split_into_words=True, return_offsets_mapping=True,
              return_overflowing_tokens=True, padding=True, truncation=True, max_length=max_length, stride=overlap)
    
    labels=encode_tags(encodings, data['tags'], max_length, overlap)
    encodings.pop("offset_mapping")
    encodings.pop("overflow_to_sample_mapping")
#     train_dataset = tf.data.Dataset.from_tensor_slices((
#         dict(train_encodings),
#         train_labels
#     ))
    return torch.utils.data.TensorDataset({'input_ids':torch.tensor(encodings['input_ids'],dtype=torch.long),
'attention_mask':torch.tensor(encodings['attention_mask'],dtype=torch.long),
'labels':torch.tensor(labels,dtype=torch.long)})
#     return torch.utils.data.TensorDataset(torch.tensor(encodings['input_ids'],dtype=torch.long),
#         torch.tensor(encodings['attention_mask'],dtype=torch.long),
#         torch.tensor(labels,dtype=torch.long))
test_dataset=process_dataset(ted,'test',10,3)
# dev_dataset=process_dataset(ted,'dev')
# train_dataset=process_dataset(ted,'train')

# for name,dataset in {'test':test_dataset}.items():# 'train':train_dataset, 'dev':dev_dataset, 
#     torch.save(dataset, './ted-'+name+'.pt')


...............................................................................................................................................................................................................................................................................................................................................................................................................

AttributeError: 'dict' object has no attribute 'size'

In [17]:
list(iter(test_dataset))[:4]

[(tensor([  101,  2627,  1303,  1110, 17136,  1118,  1297,  1223,  1103,   102]),
  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
  tensor([   0,    0,    0,    0,    0,    0,    0,    0,    0, -100])),
 (tensor([  101,  1297,  1223,  1103,  2343, 19420,  1986,  1184,  1225,   102]),
  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
  tensor([   0,    0,    0,    0,    7,    4,    2,    0,    0, -100])),
 (tensor([ 101, 1986, 1184, 1225, 1195, 1198, 1202, 2421,  112,  102]),
  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
  tensor([   4,    2,    0,    0,    0,    0,    7,    0, -100, -100])),
 (tensor([  101,  1202,  2421,   112,   188,  4267, 11553,  5822,  1142,   102]),
  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
  tensor([   0,    7,    0, -100, -100,    0, -100, -100,    0, -100]))]

In [112]:
try:
    # Disable all GPUS
    tf.config.set_visible_devices([], 'GPU')
    visible_devices = tf.config.get_visible_devices()
    for device in visible_devices:
        assert device.device_type != 'GPU'
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

In [118]:
import tensorflow as tf

# Set the offset as 1
# Set the cls tag as offset-1 value
# Set the non zeros with the offset to sum(nonzeros)
# offset+= sum(nonzeros in first half)
# stop when offset+ len(mapping)>len(tag)


def dataset2tf(dataset,split):
    data=dataset[split].map(chunk_examples_with_degree(0), batched=True, batch_size=10,remove_columns=dataset[split].column_names)
    train_encodings=tokenizer(data['texts'], is_split_into_words=True, return_offsets_mapping=True, 
              return_overflowing_tokens=True, padding=True, truncation=True, max_length=32, stride=15,)

    train_labels=encode_tags(train_encodings, data['tags'])
    train_encodings.pop("offset_mapping")
    train_dataset = tf.data.Dataset.from_tensor_slices((
        dict(train_encodings),
        train_labels
    ))
    return train_dataset
test_dataset=dataset2tf(ted,'test')
    

Loading cached processed dataset at /home/nxingyu/.cache/huggingface/datasets/csv/default-295ea44b803e5492/0.0.0/2960f95a26e85d40ca41a230ac88787f715ee3003edaacb8b1f0891e9f04dda2/cache-47a2b754535508b1.arrow


.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.


In [13]:
for i in test_dataset.take(4):
    print(i)

AttributeError: 'TensorDataset' object has no attribute 'take'

In [72]:
train_encodings[146]

Encoding(num_tokens=32, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [80]:
# [len(data[i]['tags']) for i in range(144,150)]
list(zip(data[145]['texts'][2700:],[id2tag[t] if t>0 else '' for t in data[145]['tags'][2701:]]))

[('was', ''),
 ('to', ''),
 ('focus', ''),
 ('on', ''),
 ('math', ''),
 ('and', ''),
 ('science', ','),
 ('to', ''),
 ('focus', ''),
 ('on', ''),
 ('basic', ''),
 ('research', '.'),
 ('And', ''),
 ("that's", ''),
 ('what', ''),
 ("we've", ''),
 ('done', '.'),
 ('Six', ''),
 ('years', ''),
 ('ago', ''),
 ('or', ''),
 ('so', ','),
 ('I', ''),
 ('left', ''),
 ('Renaissance', ''),
 ('and', ''),
 ('went', ''),
 ('to', ''),
 ('work', ''),
 ('at', ''),
 ('the', ''),
 ('foundation', '.'),
 ('So', ''),
 ("that's", ''),
 ('what', ''),
 ('we', ''),
 ('do', '.'),
 ('And', ''),
 ('so', ''),
 ('Math', ''),
 ('for', ''),
 ('America', ''),
 ('is', ''),
 ('basically', ''),
 ('investing', ''),
 ('in', ''),
 ('math', ''),
 ('teachers', ''),
 ('around', ''),
 ('the', ''),
 ('country', ','),
 ('giving', ''),
 ('them', ''),
 ('some', ''),
 ('extra', ''),
 ('income', ','),
 ('giving', ''),
 ('them', ''),
 ('support', ''),
 ('and', ''),
 ('coaching', '.'),
 ('And', ''),
 ('really', ''),
 ('trying', ''),
 ('to

In [ ]:
# chunked_ted_val=ted['val'].select([0,1]).map(chunk_examples_with_degree(0), batched=True, batch_size=10,remove_columns=ted['train'].column_names)
# encodings=tokenizer(chunked_ted['texts'], is_split_into_words=True, return_offsets_mapping=True, 
#           return_overflowing_tokens=True, padding=True, truncation=True, max_length=32, stride=15,)
# labels=encode_tags(encodings, chunked_ted['tags'])
# encodings.pop("offset_mapping")
# dataset = tf.data.Dataset.from_tensor_slices((
#     dict(encodings),
#     labels
# ))
# val_dataset

In [124]:
from transformers import TFDistilBertForTokenClassification
model = TFDistilBertForTokenClassification.from_pretrained('distilbert-base-cased', num_labels=len(tags)+1)

Some layers from the model checkpoint at distilbert-base-cased were not used when initializing TFDistilBertForTokenClassification: ['activation_13', 'vocab_projector', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier', 'dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [125]:
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss) # can also use any keras loss fn
model.fit(test_dataset.batch(16), epochs=3, batch_size=16, validation_data=test_dataset,verbose=1)

Model: "tf_distil_bert_for_token_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
distilbert (TFDistilBertMain multiple                  65190912  
_________________________________________________________________
dropout_39 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  7690      
Total params: 65,198,602
Trainable params: 65,198,602
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
3310/3310 [==============================] - ETA: 0s - loss: 0.1861

KeyboardInterrupt: 

In [27]:
sample_output=model.predict(dataset)

In [47]:
sample_output['logits'].argmax(axis=2).reshape(32,-1)

array([[0, 0, 0, ..., 2, 0, 0],
       [5, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 2],
       ...,
       [0, 0, 0, ..., 5, 0, 0],
       [0, 0, 0, ..., 0, 5, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [112]:
tokenizer(chunked_ted['texts'][0], is_split_into_words=True, return_offsets_mapping=True, return_overflowing_tokens=True, padding=True, truncation=True, stride=8, max_length=16)

{'input_ids': [[101, 1448, 2247, 4427, 1107, 1381, 5227, 2021, 15474, 8449, 1105, 8703, 170, 1299, 1150, 102], [101, 2021, 15474, 8449, 1105, 8703, 170, 1299, 1150, 1691, 10108, 102, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]], 'offset_mapping': [[(0, 0), (0, 3), (0, 6), (0, 9), (0, 2), (0, 4), (0, 2), (0, 6), (0, 8), (0, 10), (0, 3), (0, 8), (0, 1), (0, 3), (0, 3), (0, 0)], [(0, 0), (0, 6), (0, 8), (0, 10), (0, 3), (0, 8), (0, 1), (0, 3), (0, 3), (0, 8), (0, 10), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]], 'overflow_to_sample_mapping': [0, 0]}

In [223]:
import tensorflow as tf
encodings.pop("offset_mapping")

dataset = tf.data.Dataset.from_tensor_slices((
    dict(encodings),
    labels
))

In [227]:
for data in dataset.take(1):
    print(data)

({'input_ids': <tf.Tensor: shape=(25,), dtype=int32, numpy=
array([  101,  1448,  2247,  4427,  1107,  1381,  5227,  2021, 15474,
        8449,  1105,  8703,   170,  1299,  1150,  1691,   102,     0,
           0,     0,     0,     0,     0,     0,     0], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(25,), dtype=int32, numpy=
array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(25,), dtype=int32, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0], dtype=int32)>}, <tf.Tensor: shape=(25,), dtype=int32, numpy=
array([-100,    0,    0,    0,    0,    2,    0,    0,    2,    0,    0,
          0,    0,    0,    0,    0, -100, -100, -100, -100, -100, -100,
       -100, -100, -100], dtype=int32)>)


In [57]:
tokenizer(ted['train'][0]['transcript'], return_offsets_mapping=True, padding=True, truncation=True)

{'input_ids': [101, 20062, 117, 1166, 122, 1553, 126, 3775, 1234, 2541, 4223, 4139, 119, 1130, 2593, 117, 1234, 1132, 2257, 1106, 10556, 1147, 1583, 117, 2128, 1166, 1405, 1550, 8940, 119, 4288, 117, 1443, 170, 4095, 117, 1132, 1103, 1211, 7386, 1105, 8018, 5256, 795, 1133, 1136, 1198, 1121, 1103, 5119, 2952, 18552, 117, 1133, 1121, 1103, 1510, 8362, 20080, 27443, 3154, 1115, 8755, 1138, 1113, 1147, 2073, 119, 1109, 5758, 1104, 1594, 1817, 1482, 1120, 170, 1842, 1344, 3187, 1111, 1103, 1718, 1104, 6438, 1105, 18560, 2645, 119, 4288, 117, 1112, 1195, 1169, 1178, 5403, 117, 1209, 1631, 4472, 117, 4963, 1105, 1120, 3187, 119, 1252, 1175, 1110, 1363, 2371, 119, 1109, 3068, 1104, 1920, 1115, 1482, 3531, 1107, 1147, 2073, 1169, 1138, 170, 1167, 2418, 2629, 1113, 1147, 1218, 118, 1217, 1190, 1121, 1103, 4315, 5758, 1104, 1594, 1115, 1152, 1138, 1151, 5490, 1106, 119, 1573, 2140, 117, 1482, 1169, 1129, 4921, 1118, 3258, 117, 5343, 6486, 1158, 1219, 1105, 1170, 4139, 119, 1130, 1349, 117, 146, 

In [50]:
#create_pretraining_data.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import random
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
import tensorflow as tf

flags = tf.compat.v1.flags
FLAGS = flags.FLAGS

In [51]:
class TrainingInstance(object):
    def __init__(self, tokens, segment_ids, masked_lm_positions, masked_lm_labels, is_random_next):
        self.tokens = tokens
        self.segment_ids = segment_ids
        self.is_random_next = is_random_next
        self.masked_lm_positions = masked_lm_positions
        self.masked_lm_labels = masked_lm_labels

<module 'tensorflow.python.platform.flags' from '/home/nxingyu/miniconda3/envs/NLP/lib/python3.8/site-packages/tensorflow/python/platform/flags.py'>

In [25]:
import logging
root_logger= logging.getLogger()
root_logger.setLevel(logging.DEBUG)
handler = logging.FileHandler('test.log', 'w', 'utf-8')
handler.setFormatter(logging.Formatter("%(asctime)s [%(levelname)s] %(message)s"))
root_logger.addHandler(handler)

import subprocess
root_logger.info(subprocess.check_output(['git', 'describe', '--always']))
import datetime
root_logger.warning(datetime.datetime.now().strftime('%Y-%m-%d %H:%M'))

2020-12-21 11:43:00,756 [INFO] b'fcfb139\n'
2020-12-21 11:43:00,761 [WARNING] 2020-12-21 11:43
